# Assignment 5 - Bonus

In this assignment, you will estimate a variation on one of the regressions of Coibion and Gorodnychenko (2015), ”Information rigidity and the expectations formation process: A simple framework and new facts” that was published in the American Economic Review 105(8), 2644-78. The regression from that paper has become very impactful in the subsequent literature/discussion on expectation formation in macroeconomics.

1. Download actual and expected inflation from FED St. Louis and FED New York (https://fred.stlouisfed.org/series/CPALTT01USM659N and https://www.newyorkfed.org/microeconomics/databank.html).
2. Read the csv files into pandas DataFrames
3. Make sure the dates are recognized as such and use them as indices.
4. The forecasts that you will need in order to calculate forecast errors and forecast revisions are in the columns ”Median one-year ahead expected inflation rate” and ”Median three-year ahead expected inflation rate”. Rename these columns to something shorter.
5. Merge your two DataFrames so that you have a single DataFrame that contains realized inflation as well as the forecasts from the survey.

In [132]:
import pandas as pd
from statsmodels.formula.api import ols

In [115]:
InflationExpectation = pd.read_csv('/Users/maxweber/Desktop/HWS21/CompMacro/ComputationalMacro/InflationExpectations.csv', header = 0, delimiter=';')

InflationExpectation['DATE'] = InflationExpectation['DATE'].astype('str')
InflationExpectation['DATE'] = pd.to_datetime(InflationExpectation['DATE'],format = '%Y%m%d')

InflationExpectation['Median one-year ahead expected inflation rate'] = InflationExpectation['Median one-year ahead expected inflation rate'].apply(lambda x: x.replace(',','.'))
InflationExpectation['Median three-year ahead expected inflation rate'] = InflationExpectation['Median three-year ahead expected inflation rate'].apply(lambda x: x.replace(',','.'))
InflationExpectation[['Median one-year ahead expected inflation rate','Median three-year ahead expected inflation rate']] = InflationExpectation[['Median one-year ahead expected inflation rate','Median three-year ahead expected inflation rate']].astype('float')

InflationExpectation = InflationExpectation.iloc[:,[0,1,2]]
print(InflationExpectation.shape)
InflationExpectation.set_index('DATE',inplace=True)
InflationExpectation.head()

(103, 3)


,Median one-year ahead expected inflation rate,Median three-year ahead expected inflation rate
DATE,,
2013-06-01,3.09,3.42
2013-07-01,3.16,3.31
2013-08-01,3.40,3.80
2013-09-01,3.37,3.55
2013-10-01,3.17,3.20


In [116]:
inflation = pd.read_csv('/Users/maxweber/Desktop/HWS21/CompMacro/ComputationalMacro/CPALTT01USM659N.csv', header = 0)
inflation['DATE'] = pd.to_datetime(inflation['DATE'])
print(inflation.shape)
inflation.set_index('DATE',inplace=True)
inflation.head()

(742, 2)


,CPALTT01USM659N
DATE,
1960-01-01,1.034483
1960-02-01,1.730104
1960-03-01,1.730104
1960-04-01,1.724138
1960-05-01,1.724138


In [126]:
#df = InflationExpectation.merge(inflation, how = 'inner', left_index=True, right_index=True)
df = pd.merge_asof(inflation, InflationExpectation, on = 'DATE')
df.set_index('DATE', inplace=True)
df.columns = ['pi', 'exp_pi1', 'exp_pi3']
df.dropna(inplace=True)
df

,pi,exp_pi1,exp_pi3
DATE,,,
2013-06-01,1.754417,3.09,3.42
2013-07-01,1.960682,3.16,3.31
2013-08-01,1.518368,3.40,3.80
2013-09-01,1.184925,3.37,3.55
2013-10-01,0.963613,3.17,3.20
...,...,...,...
2021-06-01,5.391451,4.80,3.55
2021-07-01,5.365475,4.84,3.71
2021-08-01,5.251272,5.18,4.00


6. Calculate forecast errors and forecast revision in the way discussed above. Make sure you get the timing right by ’shifting’ different columns by the right amount and in the right direction.
7. Drop all columns from your final DataFrame apart from the two columns with forecast revisions and forecast errors that you just created.
8. Drop all missing observations from your final DataFrame (Observations where NaN is written). Now your DataFrame is ready for the regression.

In [138]:
#create exp_pi1_lag and exp_pi3_lag bei shifting down 12;36 rows respectively
df['exp_pi1_lag'] = df['exp_pi1'].shift(12)
df['exp_pi3_lag'] = df['exp_pi1'].shift(36)

# drop nas since we cant calculate errors for those periods
df.dropna(inplace=True)

# calculate forecast errors frcst_err_1 and frcst_err_3
df['frcst_err_1'] = df['pi'] - df['exp_pi1']
df['frcst_err_3'] = df['pi'] - df['exp_pi3']
df.head()

,pi,exp_pi1,exp_pi3,exp_pi1_lag,exp_pi3_lag,frcst_err_1,frcst_err_3
DATE,,,,,,,
2021-06-01,5.391451,4.80,3.55,2.68,2.98,0.591451,1.841451
2021-07-01,5.365475,4.84,3.71,2.89,2.98,0.525475,1.655475
2021-08-01,5.251272,5.18,4.00,3.00,3.00,0.071272,1.251272
2021-09-01,5.390349,5.31,4.19,2.98,3.00,0.080349,1.200349
2021-10-01,6.221869,5.65,4.21,2.84,3.00,0.571869,2.011869


In [139]:
df_final = df[['exp_pi1', 'exp_pi3', 'frcst_err_1', 'frcst_err_3']]
df_final.head()

,exp_pi1,exp_pi3,frcst_err_1,frcst_err_3
DATE,,,,
2021-06-01,4.80,3.55,0.591451,1.841451
2021-07-01,4.84,3.71,0.525475,1.655475
2021-08-01,5.18,4.00,0.071272,1.251272
2021-09-01,5.31,4.19,0.080349,1.200349
2021-10-01,5.65,4.21,0.571869,2.011869


9. Estimate equation (1) using your final DataFrame. You can use the statsmodels ols function for this as in the lecture. In order to stay close to the estimation of Coibion and Gorodnychenko you should estimate the equation with standard errors that allow for heteroskedasticity and autocorrelation. To do this, you can add the following as arguments to the .fit() method: cov_type = "HAC"; cov_kwds = {'maxlags' : 3}
10. Are your results in line with the above mentioned finding of Coibion and Gorodnychenko (2015)? Give at least two possible interpretations of (or explanations for) the result that you obtained.

In [140]:
model = ols(formula = 'frcst_err_1 ~ frcst_err_3', data = df_final)
fit = model.fit(cov_type = "HAC", cov_kwds = {'maxlags' : 3})
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:            frcst_err_1   R-squared:                       0.907
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     77.64
Date:                Sat, 29 Jan 2022   Prob (F-statistic):            0.00308
Time:                        18:07:38   Log-Likelihood:                 5.9815
No. Observations:                   5   AIC:                            -7.963
Df Residuals:                       3   BIC:                            -8.744
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.7671      0.103     -7.437      

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


Forecast errors are positive when real inflation is larger than the expectationw was.

c = -0.7671
-> on average, the 1 year error is lower than the 3 year error.
beta = 0.7130
-> the forecast errors are positivly correlated.

The findings are consistent 